In [56]:
import numpy as np
import pandas as pd
import datetime
import pytz
import wordle_cheater as wrdlc
from IPython import display

In [3]:
wdl = wrdlc.wordl()
wdl.add_word('alert',[3,3,2,3,3])
wdl.add_word('noisy',[2,3,1,3,3])
df = wrdlc.filter_words(wdl.try_it()) 
df.probability = df.probability.round(5)
df

,word,count,probability
1,being,242783091,0.95152
2,knife,11297715,0.04428
3,whine,832743,0.00326
4,feign,116753,0.00046
5,deign,94806,0.00037
6,eking,26695,0.00010


In [28]:
td_word = wdl.todays_word

In [46]:
def get_letter_list(your_word,the_word):
    nums =[]
    for i,l in enumerate(list(your_word)):
        if l == the_word[i]:
            nums.append(1)
        elif l in the_word:
            nums.append(2)
        else:
            nums.append(3)
    return nums    
        
def letter_list_to_str(letter_list):
    return ''.join(
        [str(v) for v in letter_list]
    )
def get_filter(your_word,the_word):
    letter_string = letter_list_to_str(get_letter_list(your_word,td_word))
    return your_word + "," + letter_string

In [54]:
initial_words = ['alert','noisy']
initial_filters = ','.join(
    np.array(
        [
            get_filter(word,get_filter(word,td_word))
            for word in initial_words
        ]
    ).reshape(-1).tolist()
)

filters = initial_filters

results_list = []
for _ in range(3):
    url = f'http://127.0.0.1:8000/wordl?filters={filters}'
    response = requests.get(url)
    resp_dict = response.json()    
    df_results = pd.DataFrame(resp_dict)
    results_list.append(df_results.copy())
    if len(df_results)<=1:
        break
    next_word = df_results.iloc[0].word
    new_filter = get_filter(next_word,td_word)
    filters = filters + "," + new_filter
    

In [58]:
for df_rl in results_list:
    display.display(df_rl)

,word,count,probability
0,shirt,40501964,0.80228
1,skirt,9981485,0.19772


,word,count,probability
0,skirt,9981485,1.0


In [33]:
url = f'http://127.0.0.1:8000/wordl?filters={filters}'
url,'http://127.0.0.1:8000/wordl?filters=alert,33233,noisy,23133'

('http://127.0.0.1:8000/wordl?filters=alert,33311,noisy,33123',
 'http://127.0.0.1:8000/wordl?filters=alert,33233,noisy,23133')

In [38]:
import requests
resp_dict

[{'word': 'shirt', 'count': 40501964, 'probability': 0.80228},
 {'word': 'skirt', 'count': 9981485, 'probability': 0.19772}]

,word,count,probability
0,shirt,40501964,0.80228
1,skirt,9981485,0.19772


In [ ]:
if